# Function Calling

Function calling is useful when are interested in using a tool that can assistant the LLM with the final response. 

LLMs today come with tool calling capabilities, allowing the LLM to call a tool when it needs assistance with the final response.

In [1]:
# Import warnings
import warnings
warnings.filterwarnings("ignore")

from utils import *
import json

Define dummy function:

In [2]:
# Defines a dummy function to get the current weather
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather = {
        "location": location,
        "temperature": "50",
        "unit": unit,
    }

    return json.dumps(weather)

### Define Functions

As demonstrated in the OpenAI documentation, here is a simple example of how to define the functions that are going to be part of the request.

The descriptions are important because these are passed directly to the LLM as part of the `system message` and the LLM will use the description to determine whether to use the functions.

In [3]:
# define a function as tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },   
    }
]

### Function Calling Best Practices

- Function names, description, and parameters should be clearly written
- Functions count towards the context limit so pay attention to # of functions and length of descriptions
- Don't make the model generate arguments you are already know 
- Keep the number of functions small for better accuracy (OpenAI recommend ~20)
- For large number of function, consider the fine-tuning option to improve accuracy and reduce costs (i.e, save on token usage)

Let's now test:

In [4]:
# define a list of messages

messages = [
    {
        "role": "user",
        "content": "What is the weather like in London?"
    }
]

In [5]:
response = get_chat_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_icqvcE7zqqUpt2yGLPX03X7p', function=Function(arguments='{"location":"London, UK"}', name='get_current_weather'), type='function')])


We can now capture the arguments:



In [6]:
args = json.loads(response.tool_calls[0].function.arguments)
print(args)

{'location': 'London, UK'}


Pass arguments to the actual function/tool:

In [7]:
get_current_weather(**args)

'{"location": "London, UK", "temperature": "50", "unit": "fahrenheit"}'

### Controlling Function Calling Behavior

Let's say we were interested in designing this `function_calling` functionality in the context of an LLM-powered conversational agent. Your solution should then know what function to call or if it needs to be called at all. Let's try a simple example of a greeting message:

In [8]:
messages = [
    {
        "role": "user",
        "content": "Hello! How are you?",
    }
]

In [9]:
get_chat_completion(messages, tools=tools)

ChatCompletionMessage(content="Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

You can specify the behavior you want from function calling, which is desired to control the behavior of your system. By default, the model decide on its own whether to call a function and which function to call. This is achieved by setting tool_choice: "auto" which is the `default` setting.

In [10]:
get_chat_completion(messages, tools=tools, tool_choice="auto")


ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

Setting tool_choice: `"none"` forces the model to not use any of the functions provided.

In [11]:
get_chat_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

Testing with user message:

In [12]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London?",
    }
]
get_chat_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content='Would you like the temperature in Celsius or Fahrenheit?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

You can also force the model to choose a function if that's the behavior you want in your application. Example:

In [13]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in London?",
    }
]
get_chat_completion(messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_current_weather"}})

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3nx7XsVPO1OfPo2n3KuEjoZK', function=Function(arguments='{"location":"London, UK"}', name='get_current_weather'), type='function')])

The OpenAI APIs also support parallel function calling that can call multiple functions in one turn. Helps with efficiency and when you have different set of distinct arguments that you want to extract.

In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in London and Belmopan?",
    }
]
get_chat_completion(messages, tools=tools)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Xbvyc6IWOp7v9WPwQuy2l9fG', function=Function(arguments='{"location": "London"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_rGRS88ukRw8EzwTSx1X3oELR', function=Function(arguments='{"location": "Belmopan"}', name='get_current_weather'), type='function')])

You can see in the response above that the response contains information from the function calls for the two locations queried.



### Function Calling Response for Model Feedback
You might also be interested in developing an agent that passes back the result obtained after calling your APIs with the inputs generated from function calling. Let's look at an example next:

In [23]:
messages = []
messages.append({"role": "user", "content": "What's the weather like in Boston!"})
assistant_message = get_chat_completion(messages, tools=tools, tool_choice="auto")
assistant_message = json.loads(assistant_message.model_dump_json())
assistant_message["content"] = str(assistant_message["tool_calls"][0]["function"])

del assistant_message["function_call"]

In [24]:
print(assistant_message)

{'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}', 'refusal': None, 'role': 'assistant', 'audio': None, 'tool_calls': [{'id': 'call_HjEsBatvMQLfyvpW3YjaRd6k', 'function': {'arguments': '{"location":"Boston, MA"}', 'name': 'get_current_weather'}, 'type': 'function'}]}


In [26]:
print(messages)

[{'role': 'user', 'content': "What's the weather like in Boston!"}]


In [27]:
messages.append(assistant_message)

In [28]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'tool_calls': [{'id': 'call_HjEsBatvMQLfyvpW3YjaRd6k',
    'function': {'arguments': '{"location":"Boston, MA"}',
     'name': 'get_current_weather'},
    'type': 'function'}]}]

We then append the results of the `get_current_weather` function and pass it back to the model using a `tool` role.


In [29]:
# get the weather information to pass back to the model
weather = get_current_weather(messages[1]["tool_calls"][0]["function"]["arguments"])

messages.append({"role": "tool",
                 "tool_call_id": assistant_message["tool_calls"][0]["id"],
                 "name": assistant_message["tool_calls"][0]["function"]["name"],
                 "content": weather})

In [30]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'tool_calls': [{'id': 'call_HjEsBatvMQLfyvpW3YjaRd6k',
    'function': {'arguments': '{"location":"Boston, MA"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_HjEsBatvMQLfyvpW3YjaRd6k',
  'name': 'get_current_weather',
  'content': '{"location": "{\\"location\\":\\"Boston, MA\\"}", "temperature": "50", "unit": "fahrenheit"}'}]

We have basically injected back the information that we obtained from the function back to the model as part of the request. The model can now use all that context to generate an appropriate response. If the function doesn't have a return value (e.g., `send_email()`) just return a success string (e.g., `"success"`).

In [31]:
final_response = get_chat_completion(messages, tools=tools)
print(final_response)

ChatCompletionMessage(content='The current temperature in Boston, MA is 50°F.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
